# references
  [*Ronneberger et al, 2015, U-Net: Convolutional Networks for Biomedical Image Segmentation*](http://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/)
  
  [*Machireddy et al, 2021, Robust Segmentation of Cellular Ultrastructure on Sparsely Labeled 3D Electron Microscopy Images using Deep Learning*](https://www.biorxiv.org/content/10.1101/2021.05.27.446019v1.full)

# objectives
- PROOF OF CONCEPT PROOF OF CONCEPT PROOF OF CONCEPT PROOF OF CONCEPT PROOF OF CONCEPT PROOF OF CONCEPT 
- Streamline the use of volumetric electron microscopy (vEM) cell images to create a 3D model for the purpose of gaining *"deeper understanding of the cellular and subcellular organization of tumor cells and their interactions with the tumor microenvironment (to) shed light on how cancer evolves and guide effective therapy choices."*
- Provide models and tools for researchers to tailor to meet their needs
- Support an interative training and test model, allowing researchers to start/stop and adjust without losing existing progress
- Take advantage of hardware acceleration

# workflow

Using electron microscopy (vEM) cell images to create a 3D model is as follows:
- Take a stack of vEM slices of tissue and 
  - pre-process the images as needed and scale them to 512x512
- If training the model
  - optionally apply data augmentation to increase the training dataset and push the images through the the training process
  - optionally, load a pretrained model and use additional training to fine-tune
  - train the model, saving checkpoints periodically and reporting progress as the model is training
  - save the final model parameters/weights
- If segmenting images with an existing model
  - create the model and load pretrained weights into the model
  - convert each image into a segmented image using a pretrained UNet model
  - save the segmented images as PNG files
- Load the stack of images as a 3D [NumPy](https://numpy.org/doc/stable/) array using [imageio.imread()](https://imageio.readthedocs.io/en/v2.16.1/_autosummary/imageio.imread.html).
- Use the [marching cubes algorithm](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.marching_cubes) from the scikit-image submodule `skimage.measure` to convert the voxels of interest to a list of faces defined by vertices on the surface of the volume.
- Use [numpy-stl](https://numpy-stl.readthedocs.io/en/latest/) to create an `stl.Mesh` object from the list of faces and vertices (as done in [this example](https://numpy-stl.readthedocs.io/en/latest/usage.html#creating-mesh-objects-from-a-list-of-vertices-and-faces)) then save the mesh with `stl.Mesh.save()`.
- As a bonus, you can use the Python package for the [Open3D](http://www.open3d.org/docs/release/) library to open & view multiple STL files (not included here)!

# dependencies

model and data use TensorFlow+Keras >= 2.12

In [11]:
import imageio
import numpy as np
from stl import Mesh
import skimage.measure
import os
import glob
import random as rand
from model import *
from data import *

# knobs and levers

edit to match your needs

In [24]:
# location of training and test data
data_dir = "data/membrane/"
data_train_dir = data_dir+"train"
data_test_dir = data_dir+"test"

# save/load model weights
model_weights_file = 'unet_membrane.hdf5'

# slice stack sythesizer randomly grabs one image from this match in data_test_dir
synth_image_stack_wildcard = "??_predict.png"

# directory and file name for eventual 3d model
stl_path = data_dir+"3d/"
stl_file = stl_path+"cube.stl"

# verbose?
chatty_mode = True

# UNet
### train new model

TAKES MANY HOURS - SKIP IF USING PRETRAINED MODEL!

augments training data, creates the zeroed model, and trains it

In [13]:
def train_new_model():
    data_gen_args = dict(rotation_range=0.2,
                        width_shift_range=0.05,
                        height_shift_range=0.05,
                        shear_range=0.05,
                        zoom_range=0.05,
                        horizontal_flip=True,
                        fill_mode='nearest')

    # Load training data generator
    train_data_gen = trainGenerator(
        2, data_train_dir, 'image', 'label', data_gen_args, save_to_dir=None)

    # Build the UNet model
    model = unet_model(print_summary=chatty_mode)

    # Fit the model using the training data generator - will save checkpoints
    model_checkpoint = ModelCheckpoint(
        model_weights_file, monitor='loss', verbose=1 if chatty_mode else 0, save_best_only=True)
    model.fit(train_data_gen, steps_per_epoch=2000,
            epochs=5, callbacks=[model_checkpoint])
    
    return model

#model = train_new_model()


### load pre-trained model

OPTIONALLY SKIP IF JUST RAN TRAINING PASS

In [14]:
model = unet_model(model_weights_file, print_summary=chatty_mode)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 256, 256, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_24[0][0]']              
                                )                                                           

### test model and save segmented results 

In [15]:
# assumes test images are in given directory and have file names <prefix>0.png, <prefix>1.png, ...
testGene = testGenerator(data_test_dir)
results = model.predict_generator(testGene,30,verbose=1 if chatty_mode else 0)

# saves segmented images to  directory with "_predict" appeneded i.e. 0_predict.png, ...
saveResult(data_test_dir,results)

C:\Users\marcs\AppData\Local\Temp\ipykernel_16404\2878309051.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(testGene,30,verbose=1 if chatty_mode else 0)


30/30 [==============================] - 6s 211ms/step


# 3D model

### PNGs -> numpy voxel

stack segmented images into 3D numpy object

In [25]:
def synth_slice_stack(dup_count=5, print_summary=False):
    
    # randomly select a segmented image
    matching_files = glob.glob(os.path.join(data_test_dir, synth_image_stack_wildcard))
    assert len(matching_files) != 0, data_test_dir+synth_image_stack_wildcard+": found no files"

    z_spacing = 10.0  # 5-10 works fine

    print (matching_files)
    image_list = []
    for file in matching_files:
        image_list.append(imageio.v2.imread(file))
    image_stack = np.stack(image_list)
    return image_stack, z_spacing

In [26]:
image_stack, z_spacing = synth_slice_stack(print_summary=chatty_mode)

['data/membrane/test\\10_predict.png', 'data/membrane/test\\11_predict.png', 'data/membrane/test\\12_predict.png', 'data/membrane/test\\13_predict.png', 'data/membrane/test\\14_predict.png', 'data/membrane/test\\15_predict.png', 'data/membrane/test\\16_predict.png', 'data/membrane/test\\17_predict.png', 'data/membrane/test\\18_predict.png', 'data/membrane/test\\19_predict.png', 'data/membrane/test\\20_predict.png', 'data/membrane/test\\21_predict.png', 'data/membrane/test\\22_predict.png', 'data/membrane/test\\23_predict.png', 'data/membrane/test\\24_predict.png', 'data/membrane/test\\25_predict.png', 'data/membrane/test\\26_predict.png', 'data/membrane/test\\27_predict.png', 'data/membrane/test\\28_predict.png', 'data/membrane/test\\29_predict.png']


### voxels -> vertices + faces

In [27]:
# convert the voxels of interest to a list of faces defined by vertices on the surface of the volume
verts, faces, _, _ = skimage.measure.marching_cubes(image_stack, 
                                                               level=None, 
                                                               spacing=(z_spacing, 1.0, 1.0),
                                                               gradient_direction='descent', 
                                                               step_size=1, 
                                                               allow_degenerate=True, 
                                                               method='lewiner', 
                                                               mask=None)

In [28]:
# print the shape of faceted data of the volume discovered by marching_cubes()
print("Slice stack verts={0} faces={1}".format(verts.shape, faces.shape))

Slice stack verts=(527170, 3) faces=(1037588, 3)


### vertices + faces -> 3D mesh -> STL file

In [29]:
# create an stl.Mesh object from the list of faces and vertices
cube = Mesh(np.zeros(faces.shape[0], dtype=Mesh.dtype))
for i,f in enumerate(faces):
    for j in range(3):
        cube.vectors[i][j] = verts[f[j],:]

# Save mesh as STL file that can be natively viewed on MacOS, Windows, Linux and can be easily 3D printed
if not os.path.exists(stl_path):
    os.makedirs(stl_path)
cube.save(stl_file)
print("3D model generated to {0}".format(stl_file))

3D model generated to data/membrane/3d/cube.stl
